In [2]:
library(foreach)
library(doParallel)
library(Matching)
library(plyr)
library(tidyverse)
library(stringr)
library(magrittr)
library(caret)
library(itecv)
# detach("package:plyr", unload=TRUE)

In [3]:
loop_test = function(setup) {
    DGP = setup$DGP
    n = setup$n
    n_folds = setup$n_folds
    training_percent = setup$training_percent
    rep = setup$rep
    model = setup$model
    
    datas = setup_data(DGP$DGP, n, training_percent, n_folds)
    estimates = datas %$% get_estimates(data, model, cv_index, test_index) 
    errors =  estimates %$% get_errors(cv_estimates, test_estimates, datas$aux_data) 
    true_selection_error = errors$true_selection_error %>%
        mutate(DGP=DGP$name, n_folds=n_folds, training_percent=training_percent, rep=rep)
    
    return(true_selection_error)
}

In [4]:
methods = list("gbm",
               "enet")
tune_grids = list(expand.grid(n.trees=seq(1,501,50), 
                              interaction.depth=c(3), 
                              shrinkage=0.2, 
                              n.minobsinnode=3),
                  expand.grid(fraction=0.5, 
                              lambda=exp(seq(-5,2,0.5))))
models = list(methods, tune_grids) %>%
    pmap(~list(method=.x, tune_grid=.y))

simulation_params = list(DGP = powers_DGPs()[1:3] %>% 
                             imap(~list(name=.y, DGP=.x)),
                         n = 100,
                         n_folds=3,
                         training_percent=0.5,
                         rep = 1:3) %>%
cross
setups = simulation_params %>%
    map(~c(.,"models"=list(models)))

In [5]:
cl = makeCluster(4)
registerDoParallel(cl)
 
base <- 4
foreach(setup = setups, 
        .combine = bind_rows,
        .packages = (.packages()))  %dopar%  
loop_test(setup) 

stopCluster(cl)

ERROR: Error in loop_test(setup): task 1 failed - "`by` can't contain join column `subject`, `fold` which is missing from LHS"
